# Multi-Modal Input Models

In [1]:
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from skimage import io
import torch
from skimage import color
from torchvision import transforms
import torchvision.models as models
import ast
import time
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import cohen_kappa_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
import pickle
from copy import deepcopy

In [2]:
if torch.cuda.is_available:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

## Transformations

In [3]:
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

## Dataset

In [4]:
class FundusDataset_MM(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with filename information.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        left_img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 3])
        right_img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 4])

        left_image = io.imread(left_img_name)
        right_image = io.imread(right_img_name)

        if len(left_image.shape) > 2 and left_image.shape[2] == 4:
            left_image = left_image[:, :, 0]
        if len(right_image.shape) > 2 and right_image.shape[2] == 4:
            right_image = right_image[:, :, 0]

        if left_image.shape[-1] != 3:
            left_image = np.repeat(left_image[:, :, np.newaxis], 3, axis=2)
        if right_image.shape[-1] != 3:
            right_image = np.repeat(right_image[:, :, np.newaxis], 3, axis=2)

        image_class = self.data_frame.iloc[idx, -1]
        image_class = ast.literal_eval(image_class)
        image_class = torch.tensor(image_class, dtype=torch.int64)
        one_hot_image_class = F.one_hot(image_class, num_classes=2).squeeze(0)

        # Tabular patient data
        age_gender = self.data_frame.iloc[idx, 1:3] # age and gender
        age_gender = torch.tensor(age_gender, dtype=torch.float32)

        if self.transform:
            left_image = self.transform(left_image)
            right_image = self.transform(right_image)

        if left_image.dim() == 2:
            left_image = left_image.unsqueeze(0)
        if right_image.dim() == 2:
            right_image = right_image.unsqueeze(0)

        sample = {'left_image': left_image, 'right_image': right_image, 'age_gender': age_gender, 'class': one_hot_image_class}

        return sample


## Dataloading

In [5]:
# Set csv file paths for training
train_info_path = r'./OIA-ODIR/Training Set/Annotation/training_annotation_filtered_OHE.csv'
valid_info_path = r'./OIA-ODIR/Off-site Test Set/Annotation/validation_annotation_filtered_OHE.csv'
test_info_path = r'./OIA-ODIR/On-site Test Set/Annotation/testing_annotation_filtered_OHE.csv'

# Set root directory paths
train_root_dir = r'./OIA-ODIR/Training Set/Images'
valid_root_dir = r'./OIA-ODIR//Off-site Test Set/Images'
test_root_dir = r'./OIA-ODIR/On-site Test Set/Images'

# Transformed dataset for left and right fundus
transformed_dataset = {
    'train': FundusDataset_MM(csv_file=train_info_path, root_dir=train_root_dir, transform=transform),
    'validate': FundusDataset_MM(csv_file=valid_info_path, root_dir=valid_root_dir, transform=transform),
    'test': FundusDataset_MM(csv_file=test_info_path, root_dir=test_root_dir, transform=transform)
}

## Model 1: Late Fusion (SUM)

In [6]:
class LateFusionSUM_MM_Resnet18(nn.Module):
    def __init__(self):
        super(LateFusionSUM_MM_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        num_image_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()
        self.fc_image = nn.Linear(num_image_features, 512)  # Fusion layer for image features
        self.fc_tabular = nn.Linear(2, 64)  # Fusion layer for tabular features
        self.fc_final = nn.Linear(512 + 64, 512)  # Fusion layer for combined features
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right, age_gender):
        left_features = self.resnet18(left)
        right_features = self.resnet18(right)
        
        # Combine image features using element-wise sum
        fused_image_features = left_features + right_features
        fused_image_features = self.fc_image(fused_image_features)
        
        # tabular features
        tabular_features = self.fc_tabular(age_gender)
        
        # Concatenate image and tabular features
        combined_features = torch.cat((fused_image_features, tabular_features), dim=1)
        combined_features = self.fc_final(combined_features)
        
        outputs = [classifier(combined_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)


## Model 2: Late Fusion (CONCAT)

In [7]:
class LateFusionCONCAT_MM_Resnet18(nn.Module):
    def __init__(self):
        super(LateFusionCONCAT_MM_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        num_image_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()  # Remove the fully connected layer
        self.fc_image = nn.Linear(num_image_features * 2, 512)  # Fusion layer
        self.fc_tabular = nn.Linear(2, 64)
        self.fc_final = nn.Linear(512 + 64, 512)
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right, age_gender):
        left_features = self.resnet18(left)
        right_features = self.resnet18(right)
        fused_image_features = torch.cat((left_features, right_features), dim=1)  # concat features late fusion
        fused_image_features = self.fc_image(fused_image_features)

        # tabular features
        tabular_features = self.fc_tabular(age_gender)
        
        # Concatenate image and tabular features
        combined_features = torch.cat((fused_image_features, tabular_features), dim=1)
        combined_features = self.fc_final(combined_features)
        
        outputs = [classifier(combined_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)

## Model 3: Late Fusion (PROD)

In [8]:
class LateFusionPROD_MM_Resnet18(nn.Module):
    def __init__(self):
        super(LateFusionPROD_MM_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        num_image_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()  # Remove the fully connected layer
        self.fc_image = nn.Linear(num_image_features, 512)  # Fusion layer
        self.fc_tabular = nn.Linear(2, 64)  # Fusion layer for tabular features
        self.fc_final = nn.Linear(512 + 64, 512)
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right, age_gender):
        left_features = self.resnet18(left)
        right_features = self.resnet18(right)
        fused_image_features = left_features * right_features  # Element-wise product fusion
        fused_image_features = self.fc_image(fused_image_features)

        # tabular features
        tabular_features = self.fc_tabular(age_gender)

        # Concatenate image and tabular features
        combined_features = torch.cat((fused_image_features, tabular_features), dim=1)
        combined_features = self.fc_final(combined_features)
        
        outputs = [classifier(combined_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)

## Model 4: Early Fusion (SUM)

In [9]:
class EarlyFusionSUM_MM_Resnet18(nn.Module):
    def __init__(self):
        super(EarlyFusionSUM_MM_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        num_image_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()  # Remove the fully connected layer
        self.fc_image = nn.Linear(num_image_features, 512)  # Fusion layer for image features
        self.fc_tabular = nn.Linear(2, 64)  # Fusion layer for tabular features
        self.fc_final = nn.Linear(512 + 64, 512)  # Fusion layer for combined features
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right, age_gender):
        summed_input = left + right # Element-wise summation
        fused_image_features = self.resnet18(summed_input)
        fused_image_features = self.fc_image(fused_image_features)
        
        # tabular features
        tabular_features = self.fc_tabular(age_gender)

        # Concatenate image and tabular features
        combined_features = torch.cat((fused_image_features, tabular_features), dim=1)
        combined_features = self.fc_final(combined_features)

        outputs = [classifier(combined_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)

## Model 5: Early Fusion (CONCAT)

In [10]:
class EarlyFusionCONCAT_MM_Resnet18(nn.Module):
    def __init__(self):
        super(EarlyFusionCONCAT_MM_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        self.resnet18.conv1 = nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False) # To accomodate concat early fusion
        num_image_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()
        self.fc_image = nn.Linear(num_image_features, 512)  # Fusion layer for image features
        self.fc_tabular = nn.Linear(2, 64)  # Fusion layer for tabular features
        self.fc_final = nn.Linear(512 + 64, 512)  # Fusion layer for combined features
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right, age_gender):
        fused_input = torch.cat((left, right), dim=1) 
        fused_image_features = self.resnet18(fused_input)
        fused_image_features = self.fc_image(fused_image_features)

        # tabular features
        tabular_features = self.fc_tabular(age_gender)

        # Concatenate image and tabular features
        combined_features = torch.cat((fused_image_features, tabular_features), dim=1)
        combined_features = self.fc_final(combined_features)

        outputs = [classifier(combined_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)


## Model 6: Early Fusion (PROD)

In [11]:
class EarlyFusionPROD_MM_Resnet18(nn.Module):
    def __init__(self):
        super(EarlyFusionPROD_MM_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        num_image_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()  # Remove the fully connected layer
        self.fc_image = nn.Linear(num_image_features, 512)  # Fusion layer for image features
        self.fc_tabular = nn.Linear(2, 64)  # Fusion layer for tabular features
        self.fc_final = nn.Linear(512 + 64, 512)  # Fusion layer for combined features
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right, age_gender):
        fused_input = left * right # Element-wise product
        fused_image_features = self.resnet18(fused_input)
        fused_image_features = self.fc_image(fused_image_features)
        
        # tabular features
        tabular_features = self.fc_tabular(age_gender)

        # Concatenate image and tabular features
        combined_features = torch.cat((fused_image_features, tabular_features), dim=1)
        combined_features = self.fc_final(combined_features)

        outputs = [classifier(combined_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)

## Training Loop

In [12]:
torch.manual_seed(42)
np.random.seed(42)

def train_model(model, dataloader, optimizer, criterion, num_epochs=10, verbose=True, save_path=None):
    acc_dict = {'train':[],'validate':[],'test':[]}
    loss_dict = {'train':[],'validate':[],'test':[]}
    kappa_dict = {'validate': [],'test':[]}
    f1_dict = {'validate': [],'test':[]}
    auc_dict = {'validate': [],'test':[]}
    best_loss = float('inf')
    best_f1 = 0
    best_kappa = 0
    best_auc = 0
    phases = ['train','validate','test']
    since = time.time()
    for i in range(num_epochs):
        epoch_since = time.time()
        print('-'*10)
        print('Epoch: {}/{}'.format(i+1, num_epochs))
        print('-'*10)
        for p in phases:
            running_correct = 0
            running_loss = 0
            running_total = 0
            if p == 'train':
                model.train()
            else:
                model.eval()
            all_probabilities_pos = []
            all_predictions = []
            all_labels = []
            all_labels_pos = []
                
            for data in dataloader[p]:
                optimizer.zero_grad()
                left_images = data['left_image'].to(device)
                right_images = data['right_image'].to(device)
                age_gender = data['age_gender'].to(device)
                labels = data['class'].to(device)
                outputs = model(left_images, right_images, age_gender)
                num_classifiers = outputs.size(1)
                classifier_losses = []
                epoch_loss = 0 
                for i in range(num_classifiers):
                    classifier_outputs = outputs[:, i, :]
                    classifier_labels = labels[:, i, :].squeeze(dim=1)  # Squeeze extra dimension
                    classifier_loss = criterion(classifier_outputs, classifier_labels.float())
                    classifier_losses.append(classifier_loss)
                
                loss = sum(classifier_losses)
                epoch_loss += loss.item()
                
                probabilities = torch.sigmoid(outputs)
                probabilities_pos = probabilities[:, 1]
                
                labels_pos = labels[:, 1]
                
                predictions = (probabilities > 0.5).int()
                
                all_probabilities_pos.extend(probabilities_pos.cpu().detach().numpy())
                all_predictions.extend(predictions.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_labels_pos.extend(labels_pos.cpu().numpy())
                
                running_correct += torch.sum(predictions == labels).item()
                running_loss += loss.item()
                running_total += labels.size(0)
                if p== 'train':
                    loss.backward()
                    optimizer.step()
                
            epoch_acc = float(running_correct/running_total)
            epoch_loss = float(running_loss/running_total)
            if verbose or (i%10 == 0):
                print('Phase:{}, epoch loss: {:.4f}'.format(p, epoch_loss))
            acc_dict[p].append(epoch_acc)
            loss_dict[p].append(epoch_loss)
            if p == 'validate':
                all_labels_flat = np.concatenate(all_labels).flatten()
                all_predictions_flat = np.concatenate(all_predictions).flatten()
                all_labels_pos_flat = np.concatenate(all_labels_pos).flatten()
                all_probabilities_pos_flat = np.concatenate(all_probabilities_pos).flatten()
                
                kappa = cohen_kappa_score(all_labels_flat, all_predictions_flat)
                f1 = f1_score(all_labels_flat, all_predictions_flat, average='macro')
                auc = roc_auc_score(all_labels_pos, all_probabilities_pos)
                
                print('Kappa: {:.4f} F1: {:.4f} AUC: {:.4f}'.format(kappa, f1, auc))
                
                kappa_dict[p].append(kappa)
                f1_dict[p].append(f1)
                auc_dict[p].append(auc)
                
                if f1 > best_f1:
                    best_f1 = f1
                    best_model_wts = deepcopy(model.state_dict())
                    best_epoch = i+1
                    best_kappa = kappa
                    best_auc = auc
                    
            if p == 'test':
                all_labels_flat = np.concatenate(all_labels).flatten()
                all_predictions_flat = np.concatenate(all_predictions).flatten()
                all_labels_pos_flat = np.concatenate(all_labels_pos).flatten()
                all_probabilities_pos_flat = np.concatenate(all_probabilities_pos).flatten()
                
                kappa = cohen_kappa_score(all_labels_flat, all_predictions_flat)
                f1 = f1_score(all_labels_flat, all_predictions_flat, average='macro')
                auc = roc_auc_score(all_labels_pos, all_probabilities_pos)
                
                print('Kappa: {:.4f} F1: {:.4f} AUC: {:.4f}'.format(kappa, f1, auc))
                
                kappa_dict[p].append(kappa)
                f1_dict[p].append(f1)
                auc_dict[p].append(auc)
                    
        if save_path:
            torch.save(best_model_wts, save_path.format(round(best_f1, 4)))
        epoch_time_elapsed = time.time() - epoch_since
        print('Epoch complete in {:.0f}m {:.0f}s'.format(epoch_time_elapsed // 60, epoch_time_elapsed % 60))
        
    time_elapsed = time.time() - since
    print('-'*10)
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Val F1: {:4f} at Epoch {}'.format(best_f1, best_epoch))
    print('Corresponding Kappa: {:4f} and AUC: {:4f}'.format(best_kappa, best_auc))

    return acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict

In [13]:
bs = 16
lr = 0.00001

# Load data for left and right fundus
dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs, shuffle=True) for x in ['train', 'validate','test']}

In [14]:
lfs_model = LateFusionSUM_MM_Resnet18()
lfs_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/LFS_MM_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/LFS_MM_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(lfs_model.parameters(), lr=lr)
lfs_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(lfs_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
lfs_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(lfs_results, file)

----------
Epoch: 1/10
----------


/state/partition1/job-45816670/ipykernel_3107421/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.2856
Phase:validate, epoch loss: 0.1665
Kappa: 0.7119 F1: 0.8559 AUC: 0.7008


/state/partition1/job-45816670/ipykernel_3107421/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.1676
Kappa: 0.7109 F1: 0.8555 AUC: 0.6743
Epoch complete in 12m 5s
----------
Epoch: 2/10
----------


...

Phase:test, epoch loss: 0.1809
Kappa: 0.7191 F1: 0.8596 AUC: 0.7158
Epoch complete in 12m 37s
----------
Epoch: 10/10
----------


/state/partition1/job-45816670/ipykernel_3107421/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.0692
Phase:validate, epoch loss: 0.1950
Kappa: 0.7141 F1: 0.8571 AUC: 0.7429


/state/partition1/job-45816670/ipykernel_3107421/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.1865
Kappa: 0.7150 F1: 0.8575 AUC: 0.7047
Epoch complete in 11m 21s
----------
Training complete in 109m 4s
Best Val F1: 0.867172 at Epoch 8
Corresponding Kappa: 0.734343 and AUC: 0.775131


In [14]:
lfc_model = LateFusionCONCAT_MM_Resnet18()
lfc_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/LFC_MM_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/LFC_MM_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(lfc_model.parameters(), lr=lr)
lfc_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(lfc_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
lfc_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(lfc_results, file)

----------
Epoch: 1/10
----------


/state/partition1/job-45816668/ipykernel_1079633/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.2113
Phase:validate, epoch loss: 0.1697
Kappa: 0.7109 F1: 0.8554 AUC: 0.6596


/state/partition1/job-45816668/ipykernel_1079633/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.1693
Kappa: 0.7092 F1: 0.8546 AUC: 0.6478
Epoch complete in 11m 4s
----------
Epoch: 2/10
----------


...

Phase:test, epoch loss: 0.1768
Kappa: 0.7131 F1: 0.8565 AUC: 0.7034
Epoch complete in 8m 43s
----------
Epoch: 10/10
----------


/state/partition1/job-45816668/ipykernel_1079633/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.0699
Phase:validate, epoch loss: 0.1863
Kappa: 0.7162 F1: 0.8581 AUC: 0.7192


/state/partition1/job-45816668/ipykernel_1079633/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.1881
Kappa: 0.7114 F1: 0.8557 AUC: 0.6823
Epoch complete in 8m 46s
----------
Training complete in 94m 16s
Best Val F1: 0.867045 at Epoch 8
Corresponding Kappa: 0.734091 and AUC: 0.752603


In [14]:
lfp_model = LateFusionPROD_MM_Resnet18()
lfp_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/LFP_MM_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/LFP_MM_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(lfp_model.parameters(), lr=lr)
lfp_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(lfp_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
lfp_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(lfp_results, file)

----------
Epoch: 1/10
----------


/state/partition1/job-45816667/ipykernel_3472411/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.3040
Phase:validate, epoch loss: 0.1719
Kappa: 0.7076 F1: 0.8538 AUC: 0.6775


/state/partition1/job-45816667/ipykernel_3472411/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.1745
Kappa: 0.7021 F1: 0.8511 AUC: 0.6483
Epoch complete in 10m 5s
----------
Epoch: 2/10
----------


...

Phase:test, epoch loss: 0.1903
Kappa: 0.7128 F1: 0.8564 AUC: 0.6976
Epoch complete in 11m 30s
----------
Epoch: 10/10
----------


/state/partition1/job-45816667/ipykernel_3472411/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.0761
Phase:validate, epoch loss: 0.1919
Kappa: 0.7152 F1: 0.8576 AUC: 0.7399


/state/partition1/job-45816667/ipykernel_3472411/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.1907
Kappa: 0.7024 F1: 0.8512 AUC: 0.7002
Epoch complete in 12m 16s
----------
Training complete in 109m 58s
Best Val F1: 0.863130 at Epoch 8
Corresponding Kappa: 0.726263 and AUC: 0.762071


In [14]:
efs_model = EarlyFusionSUM_MM_Resnet18()
efs_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/EFS_MM_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/EFS_MM_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(efs_model.parameters(), lr=lr)
efs_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(efs_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
efs_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(efs_results, file)

----------
Epoch: 1/10
----------


/state/partition1/job-45816665/ipykernel_2983952/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.3076
Phase:validate, epoch loss: 0.1724
Kappa: 0.7088 F1: 0.8544 AUC: 0.6653


/state/partition1/job-45816665/ipykernel_2983952/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.1723
Kappa: 0.7114 F1: 0.8557 AUC: 0.6466
Epoch complete in 10m 41s
----------
Epoch: 2/10
----------


...

Phase:test, epoch loss: 0.2083
Kappa: 0.6896 F1: 0.8448 AUC: 0.6863
Epoch complete in 10m 21s
----------
Epoch: 10/10
----------


/state/partition1/job-45816665/ipykernel_2983952/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.0760
Phase:validate, epoch loss: 0.2076
Kappa: 0.6949 F1: 0.8475 AUC: 0.7043


/state/partition1/job-45816665/ipykernel_2983952/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.2148
Kappa: 0.6937 F1: 0.8468 AUC: 0.6851
Epoch complete in 10m 27s
----------
Training complete in 103m 50s
Best Val F1: 0.865152 at Epoch 8
Corresponding Kappa: 0.730303 and AUC: 0.748843


In [14]:
efc_model = EarlyFusionCONCAT_MM_Resnet18()
efc_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/EFC_MM_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/EFC_MM_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(efc_model.parameters(), lr=lr)
efc_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(efc_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
efc_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(efc_results, file)

----------
Epoch: 1/10
----------


/state/partition1/job-45816666/ipykernel_557996/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.2027
Phase:validate, epoch loss: 0.1779
Kappa: 0.7114 F1: 0.8557 AUC: 0.6203


/state/partition1/job-45816666/ipykernel_557996/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.1775
Kappa: 0.7094 F1: 0.8547 AUC: 0.6127
Epoch complete in 9m 58s
----------
Epoch: 2/10
----------


...

Phase:test, epoch loss: 0.1818
Kappa: 0.6943 F1: 0.8472 AUC: 0.6332
Epoch complete in 9m 50s
----------
Epoch: 10/10
----------


/state/partition1/job-45816666/ipykernel_557996/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.0984
Phase:validate, epoch loss: 0.1845
Kappa: 0.6932 F1: 0.8466 AUC: 0.6566


/state/partition1/job-45816666/ipykernel_557996/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.1946
Kappa: 0.6797 F1: 0.8398 AUC: 0.6131
Epoch complete in 10m 26s
----------
Training complete in 104m 28s
Best Val F1: 0.857954 at Epoch 8
Corresponding Kappa: 0.715909 and AUC: 0.694188


In [14]:
efp_model = EarlyFusionPROD_MM_Resnet18()
efp_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/EFP_MM_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/EFP_MM_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(efp_model.parameters(), lr=lr)
efp_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(efp_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
efp_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(efp_results, file)

----------
Epoch: 1/10
----------


/state/partition1/job-45816669/ipykernel_1079438/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.3085
Phase:validate, epoch loss: 0.1745
Kappa: 0.7098 F1: 0.8549 AUC: 0.6292


/state/partition1/job-45816669/ipykernel_1079438/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.1743
Kappa: 0.7075 F1: 0.8538 AUC: 0.6044
Epoch complete in 11m 1s
----------
Epoch: 2/10
----------


...

Phase:test, epoch loss: 0.2023
Kappa: 0.6874 F1: 0.8437 AUC: 0.6437
Epoch complete in 8m 20s
----------
Epoch: 10/10
----------


/state/partition1/job-45816669/ipykernel_1079438/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:train, epoch loss: 0.0800
Phase:validate, epoch loss: 0.2117
Kappa: 0.7008 F1: 0.8504 AUC: 0.7032


/state/partition1/job-45816669/ipykernel_1079438/3600892822.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_gender = torch.tensor(age_gender, dtype=torch.float32)


Phase:test, epoch loss: 0.2234
Kappa: 0.6786 F1: 0.8393 AUC: 0.6341
Epoch complete in 8m 8s
----------
Training complete in 86m 32s
Best Val F1: 0.862752 at Epoch 8
Corresponding Kappa: 0.725505 and AUC: 0.709748
